In [ ]:
import numpy as np
import sys
import os
import pandas as pd
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from levenberg_marquadt import levenberg_marquadt
from normalize import  MinMaxNormalizer
from loss_fn_tarefa4 import make_mse_loss_for_neuron
from activations_fn import tanh_derivative
from network import neuron

pd.set_option('display.float_format', '{:.5f}'.format)

# Ajuste de curva por otimização

## Carregar os dados

In [ ]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho4dados.xlsx')

df.head()

## Calcular as funções de perda

### Funções utilizadas para os cálculos

In [ ]:
def make_residuals_fn(X1: np.ndarray, X2: np.ndarray, Y: np.ndarray, activation_fn=np.tanh):
    def residuals_fn(w: np.ndarray) -> np.ndarray:
        predictions = neuron(X1, X2, weights=w, activation_fn=activation_fn)
        return Y - predictions
    return residuals_fn


def make_jacobian_fn(X1, X2, activation_fn=np.tanh, activation_deriv=tanh_derivative):
    """
    Jacobiana genérica: derivada dos resíduos em relação aos pesos.
    Funciona para qualquer função de ativação.
    """
    def jacobian_fn(w):
        X1_ = np.atleast_1d(X1)
        X2_ = np.atleast_1d(X2)
        X = np.stack([X1_, X2_, np.ones_like(X1_)], axis=1)
        z = X @ w
        phi_prime = activation_deriv(z)
        J = -phi_prime[:, np.newaxis] * X  # cada linha: -φ'(z_i) * [x1, x2, 1]
        return J
    return jacobian_fn

### Pré-processamento dos dados

In [ ]:

features = df[['x1', 'x2']]
y = df['y']

# Criar os objetos para Padronização
scaler = MinMaxNormalizer(-1, 1)
scaler_y = MinMaxNormalizer(-1, 1)

# Cria as cópias dos dados para padronizar
scaled_features = features.copy()
scaled_y = y.copy()

# Ajusta os padronizadores aos dados
scaler.fit(scaled_features)
scaler_y.fit(scaled_y.to_frame())

# Padroniza os dados
scaled_features = scaler.normalize(scaled_features)
scaled_y = scaler_y.normalize(scaled_y.to_frame()).squeeze()

# initial_weights = np.random.randn(features.shape[1] + 1)  # 2 entradas + 1 bias
initial_weights = np.zeros(features.shape[1] + 1)  # 2 entradas + 1 bias
n_iterations = 10000
tolerance = 1e-6
alpha = 1e-3

### Rodar os experimentos

In [ ]:
dict_results = {}

x1_scaled = scaled_features['x1'].values
x2_scaled = scaled_features['x2'].values
y_scaled_data = scaled_y.values

# Função de custo e gradiente
loss_function, grad_loss_function = make_mse_loss_for_neuron(x1_scaled, x2_scaled, y_scaled_data, activation_fn=np.tanh)

# Função de residuos e jacobiana
residuals_fn = make_residuals_fn(x1_scaled, x2_scaled, y_scaled_data, activation_fn=np.tanh)
jacobian_fn = make_jacobian_fn(x1_scaled, x2_scaled, activation_fn=np.tanh, activation_deriv=tanh_derivative)

# Treinar o modelo com Levenberg-Marquadt
weights, losses, n_iters = levenberg_marquadt(
    initial_weights, residuals_fn, loss_function, jacobian_fn,
    alpha=alpha, alpha_variability=10, max_iter=n_iterations,
    tolerance=tolerance, stopping_criteria=[1, 3]
)

# Desnormalizar os pesos finais
final_raw_weights = weights[-1].copy()
final_denorm_weights = scaler.desnormalize_weights(weights[-1])

# Usar o neurônio com pesos desnormalizados
x1_data = features['x1'].values
x2_data = features['x2'].values
y_hat = neuron(x1_data, x2_data, weights=final_denorm_weights, activation_fn=np.tanh)

# Cálculo das métricas finais
mse_final = np.mean((y - y_hat) ** 2)
rmse_final = np.sqrt(mse_final)
mae_final = np.mean(np.abs(y - y_hat))

dict_results = {
    'Feature_Set': "MinMax(-1,1)",
    'Loss_Function': "MSE",
    'Initial_Weights': str([f"{w:.5f}" for w in initial_weights.tolist()]),
    'Raw_Weights': str([f"{w:.5f}" for w in final_raw_weights.tolist()]),
    'Final_Weights': str([f"{w:.5f}" for w in final_denorm_weights.tolist()]),
    'Final_Loss': losses[-1],
    'MSE_Final': mse_final,
    'RMSE_Final': rmse_final,
    'MAE_Final': mae_final,
    'Iterations': n_iters
}

# Criar DataFrame com um único registro
df_result = pd.DataFrame([dict_results])  # Note os colchetes extras aqui
df_result

## Resultados

In [ ]:
# Criar figura 3D melhorada
fig = go.Figure()

# Adicionar superfície de predição para melhor visualização
x1_range = np.linspace(min(x1_data), max(x1_data), 30)
x2_range = np.linspace(min(x2_data), max(x2_data), 30)
x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)
y_grid = np.zeros_like(x1_grid)

# Calcular valores preditos para toda a superfície
for i in range(x1_grid.shape[0]):
    for j in range(x1_grid.shape[1]):
        y_grid[i, j] = neuron(x1_grid[i, j], x2_grid[i, j], weights=final_denorm_weights, activation_fn=np.tanh)

# Adicionar superfície de predição
fig.add_trace(go.Surface(
    x=x1_grid, y=x2_grid, z=y_grid,
    colorscale='Blues',
    opacity=0.7,
    showscale=False,
    name='Superfície de Predição'
))

# Adicionar pontos originais
fig.add_trace(go.Scatter3d(
    x=x1_data,
    y=x2_data,
    z=y,
    mode='markers',
    marker=dict(
        size=7,
        color='red',
        symbol='circle',
        line=dict(width=1, color='darkred')
    ),
    name='Dados Originais'
))

# Adicionar pontos previstos
fig.add_trace(go.Scatter3d(
    x=x1_data,
    y=x2_data,
    z=y_hat,
    mode='markers',
    marker=dict(
        size=7,
        color='blue',
        symbol='diamond',
        line=dict(width=1, color='darkblue')
    ),
    name='Valores Previstos'
))

# Adicionar linhas verticais mais finas
for i in range(len(x1_data)):
    fig.add_trace(go.Scatter3d(
        x=[x1_data[i], x1_data[i]],
        y=[x2_data[i], x2_data[i]],
        z=[y[i], y_hat[i]],
        mode='lines',
        line=dict(color='rgba(0,100,0,0.5)', width=2),
        showlegend=False
    ))

# Configurar layout
fig.update_layout(
    title='Modelo do Neurônio: Valores Reais vs. Previstos',
    scene=dict(
        xaxis_title='x1',
        yaxis_title='x2',
        zaxis_title='y',
        # Usar proporções cúbicas para evitar achatamento
        aspectmode='cube',
        camera=dict(
            eye=dict(x=1.5, y=-1.5, z=1),  # Posição da câmera
            up=dict(x=0, y=0, z=1)  # Orientação "para cima"
        ),
        xaxis=dict(gridcolor='lightgray'),
        yaxis=dict(gridcolor='lightgray'),
        zaxis=dict(gridcolor='lightgray')
    ),
    width=1000,
    height=800,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1
    )
)

pesos_info = (f"<b>Neurônio com tanh:</b> w=[{', '.join([f'{w:.4f}' for w in final_denorm_weights])}]<br>"
              f"MSE: {mse_final:.5f}, RMSE: {rmse_final:.5f}, MAE: {mae_final:.5f}<br>"
              f"Iterações: {n_iters}")

fig.add_annotation(
    x=0.5, y=0.02,
    xref="paper", yref="paper",
    text=pesos_info,
    showarrow=False,
    font=dict(size=13),
    bgcolor="rgba(255, 255, 255, 0.9)",
    bordercolor="black",
    borderwidth=1,
    borderpad=5
)

fig.show()